# Two tier anomally detection model 
Tier 1: Safe margin calculation  
    Upper: SM(t) = Q_h + k*STD 
    Lower: SM(t) = Q_h - k*STD 
    
Tier 2: Standard Limit calculation
    T_max, T_min According to algorithm 1 from TDSC paper 

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
import random
import importlib
import os
import sys
import json
import time
import importlib

import numpy as np
import pickle5 as pickle
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import matplotlib.dates as md

from pprint import pprint
from copy import deepcopy
from scipy.stats import hmean
from matplotlib.lines import Line2D
from tqdm.notebook import tqdm
random.seed()

In [5]:
sys.path.append("..")
from src.common_functions import *

In [6]:
clustering_version = 'maxr085'

In [7]:
# Confirm directories are in place
if not os.path.exists(os.path.join(os.getcwd(), '../data')):
    raise OSError("Must first download data, see README.md")
data_dir = os.path.join(os.getcwd(), '../data')

if not os.path.exists(os.path.join(data_dir, 'generated_clusters')):
    os.mkdir(os.path.join(data_dir, 'generated_clusters'))
cluster_dir = os.path.join(data_dir, 'generated_clusters')

if not os.path.exists(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/cleaned')):
    os.mkdir(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/cleaned'))
cleaned_dir = os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/cleaned')

if not os.path.exists(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents')):
    os.mkdir(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents'))
incidents_dir = os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents')

if not os.path.exists(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents_GT')):
    os.mkdir(os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents_GT'))
incidents_GT_dir = os.path.join(cluster_dir, f'{clustering_version}_incident_ratios/incidents_GT')

if not os.path.exists(os.path.join(data_dir, f'{clustering_version}_results')):
    os.mkdir(os.path.join(data_dir, f'{clustering_version}_results'))
results = os.path.join(data_dir, f'{clustering_version}_results')

In [8]:
results

'/home/jovyan/work/training/../data/maxr085_results'

In [9]:
print(cleaned_dir)
files = os.listdir(cleaned_dir)
pprint(files)

/home/jovyan/work/training/../data/generated_clusters/maxr085_incident_ratios/cleaned
['01_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '08_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '11_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '06_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '05_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '10_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '03_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '04_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '12_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '02_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '07_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl',
 '09_2019_ratios_maxr085_gran_5_incidents_cleaned.pkl']


# Parameters
* Be sure to run this at the start

In [10]:
start_time = '06:00'
end_time   = '20:55'
training_months = (0, 8) # January to August
cross_validation_months = (9, 10) # September and October
testing_months = (11, 12) # November and December'
months = {'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5,
          'june': 6, 'july': 7, 'august': 8, 'september': 9, 'october': 10,
          'november': 11, 'december': 12}

# Entire Year Data 
First divide into training and testing to do the rest calculation

In [11]:
info_ratio = []
i = 0
while i< len(files):
    fp = os.path.join(cleaned_dir, files[i])
    with open(fp, 'rb') as handle:
        info_ratio.append( pickle.load(handle))
    i+=1
print(len(info_ratio))
combined_ratio_frame = pd.concat(info_ratio)
print(len(combined_ratio_frame))

12
105120


In [12]:
combined_ratio_frame = combined_ratio_frame.between_time(start_time, end_time)
combined_ratio_frame =  combined_ratio_frame[(combined_ratio_frame.index.month >= months['january']) & (combined_ratio_frame.index.month <= months['august'])]
print(len(combined_ratio_frame))
training = combined_ratio_frame

43740


# Select Clusters here
* Right now it selects the 25 clusters with the most incidents throughout 2019

In [14]:
lower_bound_correlation = 0.7
correlation_threshold = 0.85

params = {'granularity': 5}
fp = os.path.join(cluster_dir, f'optimized_clustering_{lower_bound_correlation}_{correlation_threshold}_{clustering_version}_restricted.pkl')
with open(fp, 'rb') as handle:
    clusters = pickle.load(handle)
print(len(clusters))    

354


In [15]:
incidents_GT_dir

'/home/jovyan/work/training/../data/generated_clusters/maxr085_incident_ratios/incidents_GT'

In [16]:
files_GT = os.listdir(incidents_GT_dir)
incident_GT = []
i = 0
while i< len(files_GT):
    fp = os.path.join(incidents_GT_dir, files_GT[i])
    with open(fp, 'rb') as handle:
        incident_GT.append( pickle.load(handle))
    i+=1
incident_GT_Frame = pd.concat(incident_GT)

In [17]:
incident_GT_Frame

,XDSegID,Total_Number_Incidents,cluster_head
time_local,,,
2019-11-07 14:35:00,429335039,1,156081773
2019-11-23 12:20:00,429335039,1,156081773
2019-11-08 06:40:00,1524588225,1,156081773
2019-11-20 18:35:00,450430897,1,156208765
2019-11-06 13:40:00,156109841,1,156276237
...,...,...,...
2019-07-04 13:40:00,1524583863,1,1524602415
2019-07-05 15:55:00,1524602972,1,1524602415
2019-07-26 18:55:00,1524646899,1,1524646899


In [19]:
# Adjust the number of clusters
NUMBER_OF_CLUSTERS = 10

In [20]:
_df = incident_GT_Frame.groupby('cluster_head').sum()\
                       .sort_values('Total_Number_Incidents', ascending=False)

# Only pick cluster_heads which are present in the {version}_clusters.pkl
_df = _df[_df.index.isin(list(clusters.keys()))].head(NUMBER_OF_CLUSTERS)
display(_df)
cluster_list = _df.index.tolist()
print(len(cluster_list))

,XDSegID,Total_Number_Incidents
cluster_head,,
429356410,182790185565,167
1524555699,170926371562,127
1524274521,104124390121,123
1524373007,140884164940,109
1524471255,115798829656,95
1524476318,100944962864,92
441419752,72379527797,88
1524518468,92504198738,86
1524476847,97754585474,85


10


# Filename generation
> Make sure you run this

In [22]:
new_filename = f"{clustering_version}_{len(cluster_list)}C_{datetime.datetime.now().strftime('%m-%d-%Y')}"
# new_filename = "0027_25C_07-28-2021"
new_filename

'maxr085_10C_11-02-2021'

In [23]:
fp = os.path.join(results, f'used_clusters_list_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(cluster_list, handle)

# First Notebook: training_residual_and_Safe_margin.ipynb
* Training on **January** to **August** data
* Requires:
    * None
* Generates:
    * `optimized_residual_train`
    * `optimized_safe_margin`

In [24]:
info_ratio = []
i = 0
files = os.listdir(cleaned_dir)
while i < len(files):
    fp = os.path.join(cleaned_dir, files[i])
    with open(fp, 'rb') as handle:
        info_ratio.append(pickle.load(handle))
    i += 1
combined_ratio_frame = pd.concat(info_ratio)

combined_ratio_frame = combined_ratio_frame.between_time(start_time, end_time)
combined_ratio_frame =  combined_ratio_frame[(combined_ratio_frame.index.month >= months['january']) & (combined_ratio_frame.index.month <= months['august'])]
training = combined_ratio_frame

training_cluster_list = training[cluster_list]
training_cluster_list.columns = cluster_list
Q_mean_list = {} # Qmean for each of the cluster 
for column in training_cluster_list:
    Q_mean_list[column] = {}
    mad = training_cluster_list[column].mad()
    std = training_cluster_list[column].std()
    median = training_cluster_list[column].median()
    grouped = training_cluster_list[column].groupby([training_cluster_list[column].index.hour,
                                                     training_cluster_list[column].index.minute])
    Q_mean = {}
    for key,group in grouped:
        Q_mean[key] = group.mean()
    Q_mean_list[column]['Q_mean'] = Q_mean
    Q_mean_list[column]['mad'] = mad
    Q_mean_list[column]['std'] = std
    Q_mean_list[column]['median'] = median

# generate safe_margin for all values of kappa
kappa_L = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
# kappa_L = list(np.arange(0.0, 3.2, 0.1))

safe_margin = {}
for key in Q_mean_list.keys():
    safe_margin[key] = {}
    for k in kappa_L:
        safe_margin[key][k] = {'upper':{},'lower':{}}
        mad = Q_mean_list[key]['std']

        Q_mean = Q_mean_list[key]['Q_mean']
        for key1 in Q_mean.keys(): 
            safe_margin[key][k]['upper'][key1] = Q_mean[key1] + mad * k
            safe_margin[key][k]['lower'][key1] = Q_mean[key1] - mad * k

fp = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(safe_margin, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Saved optimized_safe_margin_{new_filename}.pkl')
    
residual = {}

for column in tqdm(training_cluster_list.columns):
    grouped = training_cluster_list[column].groupby([training_cluster_list[column].index.hour,
                                                     training_cluster_list[column].index.minute])
    sm_per_C = safe_margin[column]
    R_per_C = {}
    for key in sm_per_C.keys():
        nabla_dict = calculate_nabla(grouped, sm_per_C[key])

        nabla_frame = pd.DataFrame(list(nabla_dict.items()),columns = ['time','nabla'])
        nabla_frame.set_index('time', inplace=True)
        SF_List = [3,5,7,9]
        RUC = {}
        for sf in SF_List:
            RUC[sf] = faster_calculate_residual(nabla_frame,sf)
        R_per_C[key] = RUC

    residual[column] = R_per_C

fp = os.path.join(results, f'optimized_residual_train_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(residual, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Saved optimized_residual_train_{new_filename}.pkl')

Saved optimized_safe_margin_maxr085_10C_11-02-2021.pkl


  0%|          | 0/10 [00:00<?, ?it/s]

# Second Notebook: standar_limit_QR.ipynb
* Calculates $\tau_{min}$ and $\tau_{max}$
* Uses the new algorithms in the utility.py
* Training on **January** to **August** data
* Requires:
    * `optimized_residual_train`
    * `optimized_safe_margin`
* Generates:
    * `optimized_standard_limit`

In [276]:
fp_residual = os.path.join(results, f'optimized_residual_train_{new_filename}.pkl')
with open(fp_residual, 'rb') as handle:
    residual = pickle.load(handle)

fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)

residual_filtered = {}
for key in residual.keys():
    if(key in cluster_list):
        residual_filtered[key] = residual[key]

safe_margin_filtered = {}
for key in safe_margin.keys():
    if(key in cluster_list):
        safe_margin_filtered[key] = safe_margin[key]

df = pd.DataFrame.from_dict(residual_filtered, orient="index").stack().to_frame()
df = pd.DataFrame(df[0].values.tolist(), index=df.index)
indices = df.index.tolist()
sf_keys = df.columns.tolist()

standard_limit = []

pbar = tqdm(total=(len(indices) * len(sf_keys)))
for index in indices:
    for sf_key in sf_keys:
        _df = pd.DataFrame.from_dict(df.loc[index][sf_key].items())
        _df = _df.rename(columns={0:'time', 1: 'nabla'})
        _df.set_index('time', inplace=True)
        T_max = calculate_tmax(_df['nabla'])
        T_min = calculate_tmin(_df['nabla'])
        temp = {'cluster_id':index[0],
                'ka ppa':index[1],
                'SF':sf_key,
                'tau_max':T_max, 'tau_min':T_min}
        standard_limit.append(temp)
        pbar.update(1)
pbar.close()

# Saving and backing up
fp = os.path.join(results, f'optimized_standard_limit_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(standard_limit, handle)
    print(f'Saved optimized_standard_limit_{new_filename}.pkl')

  0%|          | 0/320 [00:00<?, ?it/s]

Saved optimized_standard_limit_maxr085_nolon_pcut01_10C_10-29-2021.pkl


# Third Notebook: cross_validate_residual.ipynb
* Cross validating on **September** and **October** data
* Requires:
    * `optimized_safe_margin`
* Generates:
    * `optimized_test_residual`

In [277]:
fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)

test_files = os.listdir(incidents_dir)
info_ratio_incidents = []
i = 0
while i< len(test_files):
    fp = os.path.join(incidents_dir, test_files[i])
    with open(fp, 'rb') as handle:
        info_ratio_incidents.append( pickle.load(handle))
    i+=1
combined_ratio_frame_incidents = pd.concat(info_ratio_incidents)

combined_ratio_frame_incidents = combined_ratio_frame_incidents.between_time(start_time, end_time)
combined_ratio_frame_incidents = combined_ratio_frame_incidents[(combined_ratio_frame_incidents.index.month >= months['september']) 
                                                              & (combined_ratio_frame_incidents.index.month <= months['october'])]

testing = combined_ratio_frame_incidents
testing_Clist = testing[cluster_list]
testing_Clist.columns = cluster_list

test_residual = {}

for column in tqdm(testing_Clist.columns):
    grouped = testing_Clist[column].groupby([testing_Clist[column].index.hour,
                                             testing_Clist[column].index.minute])
    sm_per_C = safe_margin[column]
    R_per_C = {}
    for key in sm_per_C.keys():
        nabla_dict = calculate_nabla(grouped, sm_per_C[key])

        nabla_frame = pd.DataFrame(list(nabla_dict.items()),columns = ['time','nabla'])
        nabla_frame.set_index('time', inplace=True)
        SF_List = [3,5,7,9]
        RUC = {}
        for sf in SF_List:
            RUC[sf] = faster_calculate_residual(nabla_frame,sf)
        R_per_C[key] = RUC

    test_residual[column] = R_per_C

# Saving and backing up
fp = os.path.join(results, f'optimized_test_residual_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(test_residual, handle)
    print(f'Saved optimized_test_residual_{new_filename}.pkl')

  0%|          | 0/10 [00:00<?, ?it/s]

Saved optimized_test_residual_maxr085_nolon_pcut01_10C_10-29-2021.pkl


# Fourth Notebook: detection_QR.ipynb
* Cross validating on **September** and **October** data
* Requires:
    * `optimized_safe_margin`
    * `optimized_standard_limit`
    * `optimized_test_residual`
* Generates:
    * `optimized_detection_report`

In [278]:
fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)

fp_standard_limit = os.path.join(results, f'optimized_standard_limit_{new_filename}.pkl')
with open(fp_standard_limit, 'rb') as handle:
    standard_limit_5C = pickle.load(handle)
standard_limit_5C_Frame = pd.DataFrame(standard_limit_5C)

fp_test_res = os.path.join(results, f'optimized_test_residual_{new_filename}.pkl')
with open(fp_test_res, 'rb') as handle:
    test_residual = pickle.load(handle)

info_ratio_incidents = []
i = 0
test_files = os.listdir(incidents_dir)
while i< len(test_files):
    fp = os.path.join(incidents_dir, test_files[i])
    with open(fp, 'rb') as handle:
        info_ratio_incidents.append( pickle.load(handle))
    i+=1
combined_ratio_frame_incidents = pd.concat(info_ratio_incidents)

testing = combined_ratio_frame_incidents.between_time(start_time, end_time)
testing =  testing[(testing.index.month >= months['september']) & (testing.index.month <= months['october']) ]
testing_Clist = testing[cluster_list]
testing_Clist.columns = cluster_list

detection_report = []
for column in tqdm(testing_Clist.columns):
    grouped = testing_Clist[column].groupby([testing_Clist[column].index.hour,
                                             testing_Clist[column].index.minute])

    sm_per_C = safe_margin[column] # safe margin list for each cluster
    for key in sm_per_C.keys(): # for each safe margin
        for key1, group in grouped:
            group = group.dropna()

            groupDF = pd.DataFrame(group)
            groupDF['g_upper'] = groupDF[column] > sm_per_C[key]['upper'][key1]
            groupDF['l_lower'] = groupDF[column] < sm_per_C[key]['lower'][key1]
            groupDF['or'] = groupDF['g_upper'] | groupDF['l_lower']

            groupDF = groupDF[groupDF['or'] == True]
            res_SF = test_residual[column][key]
            for key2 in res_SF.keys():
                std_limit = standard_limit_5C_Frame[(standard_limit_5C_Frame['cluster_id']== column) &
                                                    (standard_limit_5C_Frame['ka ppa']== key) &
                                                    (standard_limit_5C_Frame['SF']== key2)]
                index_ar = std_limit.index
                for index, row in groupDF.iterrows():
                    temp = None
                    if(res_SF[key2][index] >0):
                        if(res_SF[key2][index]>std_limit.at[index_ar[0],'tau_max']):
                            temp = {'cluster_id':column,'kappa':key,'SF':key2,
                                    'time':index,'RUC':res_SF[key2][index],'tau_max':std_limit.at[index_ar[0],'tau_max']}
                            detection_report.append(temp)
                    else:
                        if(res_SF[key2][index]<std_limit.at[index_ar[0],'tau_min']):
                            temp = {'cluster_id':column,'kappa':key,'SF':key2,'time':index,
                                    'RUC':res_SF[key2][index],'tau_min':std_limit.at[index_ar[0],'tau_min']}
                            detection_report.append(temp)

detection_report_Frame = pd.DataFrame(detection_report)
detection_report_Frame.set_index('time',inplace = True)

# Saving and backing up
fp = os.path.join(results, f"optimized_detection_report_{new_filename}.pkl")
detection_report_Frame.to_pickle(fp)
print(f"Saved optimized_detection_report_{new_filename}.pkl")

  0%|          | 0/10 [00:00<?, ?it/s]

Saved optimized_detection_report_maxr085_nolon_pcut01_10C_10-29-2021.pkl


# Fifth Notebook: analyse_detection_QR.ipynb
* Cross validating on **September** and **October** data
* Requires:
    * `optimized_detection_report`
* Generates:
    * `optimized_hyper_mapping`
    * `optimized_actual_detection_Frame`: For use with graphing

In [293]:
fp_detection_report = os.path.join(results, f"optimized_detection_report_{new_filename}.pkl")
with open(fp_detection_report, 'rb') as handle:
    detection_report_Frame = pickle.load(handle)

testing_incident_GT = incident_GT_Frame.between_time(start_time, end_time)
testing_incident_GT = testing_incident_GT[(testing_incident_GT.index.month >= months['september']) & (testing_incident_GT.index.month <= months['october'])]
testing_incident_GT_Clist = testing_incident_GT[testing_incident_GT['cluster_head'].isin(cluster_list)]

group_detection_report_by_cluster_id = detection_report_Frame.groupby('cluster_id')
group_gt_incident_cluster_head = testing_incident_GT_Clist.groupby('cluster_head')
actual_detection = []
detection_GT = []
for key, gorup in tqdm(group_detection_report_by_cluster_id):
    group_by_kappa_sf = gorup.groupby(['kappa','SF'])
    for (key1,key2), group in group_by_kappa_sf:
        for index,row in group.iterrows():
            detection_type = 0
            if key in group_gt_incident_cluster_head.groups.keys():
                for index1,row1 in group_gt_incident_cluster_head.get_group(key).iterrows():
                    #iterate only incidents happend for the cluster 
                    if((index.month == index1.month) and (index.day == index1.day)):
                        #This means incident and detection are on the same day
                        if((index.hour >= (index1.hour-2)) & (index.hour <= (index1.hour+2))):
                            #this means successful detection of the incident
                            detection_type = 1 # this is due to an Incident 
                            temp1 = {'cluster_id':key,'kappa':key1,'SF':key2,'time':index1}
                            detection_GT.append(temp1)
                        elif((index.hour >= 6) & (index.hour <= 10) or 
                            (index.hour >= 16) & (index.hour <= 18)): # this condition is for busy hour 
                            #this means detected an incident
                            detection_type = 2
                        else: # this is most probably false alarm 
                            detection_type =3
                        break
                temp = {'cluster_id':key,'kappa':key1,'SF':key2,'time':index,'detection_type':detection_type}
                actual_detection.append(temp)

actual_detection_Frame = pd.DataFrame(actual_detection)
actual_detection_Frame.set_index('time',inplace = True)
detection_GT_Frame = pd.DataFrame(detection_GT)
detection_GT_Frame.set_index('time',inplace = True)

actual_detection_Frame['detection_number'] = 0
group_actual_detection_Frame = actual_detection_Frame.groupby(['cluster_id'])
for key1, group in group_actual_detection_Frame:
    group_c = group.groupby(['kappa','SF'])
    for (key2, key3), grp in group_c:
        current = None
        detection = 0
        grp.sort_index(inplace=True)
        for index,item in grp.iterrows():
            if((current == None)):
                current = index
            else:
                if((current.month == index.month) & (current.day == index.day)):
                    if(current.hour == index.hour):
                        diff = index.minute - current.minute
                        if(diff == 5):
                            grp.at[index,'detection_number'] = detection
                            current = index
                            continue
                        else:
                            detection = detection + 1
                    else:
                        H_diff = index.hour - current.hour
                        if(H_diff == 1):
                            if((index.minute  == 0) & (current.minute == 55)):
                                grp.at[index,'detection_number'] = detection
                                current = index
                                continue
                            else:
                                detection = detection + 1
                        else:
                            detection = detection + 1
                else: 
                    detection = detection + 1
                grp.at[index,'detection_number'] = detection
                current = index
        for index,item in grp.iterrows():
            actual_detection_Frame.at[index,'detection_number'] = item.detection_number

hyper_mapping = {}
group_actual_detection_Frame = actual_detection_Frame.groupby(['cluster_id'])
for key1, group in group_actual_detection_Frame:
    group_c = group.groupby(['kappa','SF'])
    min_fa =  sys.maxsize
    min_decision_fa =  (-1.0)*sys.maxsize
    total_incident = len(testing_incident_GT_Clist[(testing_incident_GT_Clist['cluster_head']==key1)])
    min_missed = sys.maxsize
    print("CLUSTER: ",key1)
    print('total incident: ',total_incident)
    for (key2,key3),grp in group_c:
        valid_detection = len(list(grp[grp['detection_type'] == 1]['detection_number'].unique())) + len(list(grp[grp['detection_type'] == 2]['detection_number'].unique()))
        total_detection = len(list(grp['detection_number'].unique()))
        false_alarm = total_detection - valid_detection
        df3 = detection_GT_Frame[(detection_GT_Frame['cluster_id']==key1)&
                                            (detection_GT_Frame['kappa']==key2)&
                                            (detection_GT_Frame['SF']==key3)]
        df3 = df3[~df3.index.duplicated(keep='first')]
        print('len(df3):',len(df3))
        detection = len(df3)
        fraction_of_detection = detection /total_incident
        print("fraction_of_detection: ",fraction_of_detection)
        missed = abs(total_incident - detection)
#         fraction_FA  = false_alarm/ total_detection
        fraction_FA = false_alarm / (false_alarm + 16560)
        print('fraction_FA: ',fraction_FA)
        decision_factor = fraction_of_detection - fraction_FA
        print('decision_factor:', decision_factor)
        if((min_decision_fa < decision_factor)):
            min_decision_fa = decision_factor
            hyper_mapping[key1] = {'kappa':key2,'SF':key3}
            print('false alarm: ',false_alarm)

print()
print(hyper_mapping)

print()

# Saving and backing up
fp = os.path.join(results, f"optimized_hyper_mapping_{new_filename}.pkl")
with open(fp, 'wb') as handle:
    pickle.dump(hyper_mapping, handle)
    print(f"Saved optimized_hyper_mapping_{new_filename}.pkl")

# Saving and backing up
fp = os.path.join(results, f"optimized_actual_detection_Frame_{new_filename}.pkl")
actual_detection_Frame.to_pickle(fp)
print(f"Saved optimized_actual_detection_Frame_{new_filename}.pkl")

  0%|          | 0/10 [00:00<?, ?it/s]

CLUSTER:  160726844
total incident:  9
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.016335016335016335
decision_factor: 0.8725538725538725
false alarm:  275
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.015165031222123104
decision_factor: 0.8737238576667657
false alarm:  255
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.01493070013681518
decision_factor: 0.8739581887520737
false alarm:  251
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.014227037323650217
decision_factor: 0.8746618515652386
false alarm:  239
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.016218142933523437
decision_factor: 0.8726707459553654
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.015574842468196409
decision_factor: 0.8733140464206924
len(df3): 8
fraction_of_detection:  0.8888888888888888
fraction_FA:  0.015223596574690771
decision_factor: 0.8736652923141981
len(df3)

In [292]:
valid_detection, total_detection


(1, 44)

# Sixth Notebook: test_residual_QR.ipynb
* Cross validating on **October**, **November**, and **December** data
* Requires:
    * `optimized_safe_margin`
    * `optimized_hyper_mapping`
* Generates:
    * `optimized_residual`: Not to be confused with `optimized_test_residual` generated in notebook three

In [280]:
fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)
safe_margin

{441550940: {0.25: {'upper': {(6, 0): 0.9544541962020947,
    (6, 5): 0.9530530295006334,
    (6, 10): 0.9388341482578245,
    (6, 15): 0.9364599023794372,
    (6, 20): 0.934072393455224,
    (6, 25): 0.9366084837252524,
    (6, 30): 0.942136841968977,
    (6, 35): 0.9366950228023706,
    (6, 40): 0.9270624574113063,
    (6, 45): 0.9195889703275195,
    (6, 50): 0.9162125890211008,
    (6, 55): 0.9166537766348354,
    (7, 0): 0.9071527016086149,
    (7, 5): 0.9111227799007695,
    (7, 10): 0.9246414381514028,
    (7, 15): 0.9326231888601815,
    (7, 20): 0.9249207624638706,
    (7, 25): 0.9136688081078944,
    (7, 30): 0.9122376863892178,
    (7, 35): 0.9071763412321338,
    (7, 40): 0.909749126957551,
    (7, 45): 0.9151670423960567,
    (7, 50): 0.9082430058932344,
    (7, 55): 0.9090655522910469,
    (8, 0): 0.9055658676049148,
    (8, 5): 0.9066986439345157,
    (8, 10): 0.9099593219959909,
    (8, 15): 0.914193629183228,
    (8, 20): 0.9178116690467586,
    (8, 25): 0.919532714652

In [281]:
fp = os.path.join(results, f"optimized_hyper_mapping_{new_filename}.pkl")
with open(fp, 'rb') as handle:
    hyper_mapping = pickle.load(handle)
hyper_mapping

{160726844: {'kappa': 0.25, 'SF': 9},
 429334708: {'kappa': 0.25, 'SF': 3},
 449617565: {'kappa': 0.25, 'SF': 3},
 449617633: {'kappa': 0.25, 'SF': 7},
 1524308595: {'kappa': 0.25, 'SF': 3},
 1524453968: {'kappa': 0.25, 'SF': 3},
 1524544056: {'kappa': 0.25, 'SF': 3}}

In [282]:
cross_validated_kappa_SF = hyper_mapping
fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)

test_data_dir_24_may = os.path.join(os.getcwd(), 'data_22_jun/incident')
test_files = os.listdir(incidents_dir)
info_ratio_incidents = []
i = 0
while i< len(test_files):
    fp = os.path.join(incidents_dir, test_files[i])
    with open(fp, 'rb') as handle:
        info_ratio_incidents.append( pickle.load(handle))
    i+=1
combined_ratio_frame_incidents = pd.concat(info_ratio_incidents)

combined_ratio_frame_incidents = combined_ratio_frame_incidents.between_time(start_time, end_time)
combined_ratio_frame_incidents = combined_ratio_frame_incidents[(combined_ratio_frame_incidents.index.month >= months['october']) 
                                                              & (combined_ratio_frame_incidents.index.month <= months['december'])]
testing = combined_ratio_frame_incidents
testing_Clist = testing[list(cross_validated_kappa_SF.keys())]
testing_Clist.columns = list(cross_validated_kappa_SF.keys())
testing_Clist.columns

test_residual = {}
for column in tqdm(testing_Clist.columns):
    grouped = testing_Clist[column].groupby([testing_Clist[column].index.hour,
                                                    testing_Clist[column].index.minute])

    sm_per_C = safe_margin[column]
    kappa = cross_validated_kappa_SF[column]['kappa']
    SF = cross_validated_kappa_SF[column]['SF']
    R_per_C = {}
    nabla_dict = calculate_nabla(grouped,sm_per_C[kappa])
    nabla_frame = pd.DataFrame(list(nabla_dict.items()),columns = ['time','nabla'])
    nabla_frame.set_index('time', inplace=True)
    _grouped = nabla_frame.groupby(nabla_frame.index.floor('D'))
    RUC = {}

    RUCsf = {}
    for k, group in _grouped:
        df = group.rolling(SF, min_periods=SF).sum()
        df[0:SF] = group[0:SF]
        _RUC = df.to_dict()['nabla']
        RUCsf.update(_RUC)

    RUC[SF] = RUCsf
    R_per_C[kappa] = RUC
    test_residual [column] = R_per_C
    
# Saving and backing up
fp = os.path.join(results, f'optimized_residual_Test_QR_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(test_residual, handle)

  0%|          | 0/7 [00:00<?, ?it/s]

# Seventh Notebook: test_analysis_QR.ipynb
* Cross validating on **October**, **November**, and **December** data
* Requires:
    * `optimized_safe_margin`
    * `optimized_standard_limit`
    * `optimized_residual_Test_QR`
* Generates:
    * `optimized_results`
    * `optimized_actual_detection_frame`
    * `optimized_detection_report_Frame`

In [283]:
time_start = time.time()

fp_safe_margin = os.path.join(results, f'optimized_safe_margin_{new_filename}.pkl')
with open(fp_safe_margin, 'rb') as handle:
    safe_margin = pickle.load(handle)

fp_standard_limit = os.path.join(results, f'optimized_standard_limit_{new_filename}.pkl')
with open(fp_standard_limit, 'rb') as handle:
    standard_limit_5C = pickle.load(handle)
standard_limit_5C_Frame = pd.DataFrame(standard_limit_5C)

fp_test_res = os.path.join(results, f'optimized_residual_Test_QR_{new_filename}.pkl')
with open(fp_test_res, 'rb') as handle:
    test_residual = pickle.load(handle)

test_files = os.listdir(incidents_dir)
info_ratio_incidents = []
i = 0
while i< len(test_files):
    fp = os.path.join(incidents_dir, test_files[i])
    with open(fp, 'rb') as handle:
        info_ratio_incidents.append( pickle.load(handle))
    i+=1
combined_ratio_frame_incidents = pd.concat(info_ratio_incidents)

files_GT = os.listdir(incidents_GT_dir)
incident_GT = []
i = 0
while i< len(files_GT):
    fp = os.path.join(incidents_GT_dir, files_GT[i])
    with open(fp, 'rb') as handle:
        incident_GT.append( pickle.load(handle))
    i+=1
incident_GT_Frame = pd.concat(incident_GT)
cross_validated_kappa_SF = hyper_mapping

testing_incident_GT = incident_GT_Frame.between_time(start_time, end_time)
testing_incident_GT_Clist =  testing_incident_GT[testing_incident_GT['cluster_head'].isin (cluster_list)]
testing_incident_GT_Clist =  testing_incident_GT_Clist[(testing_incident_GT_Clist.index.month >= months['october']) 
                                                     & (testing_incident_GT_Clist.index.month <= months['december'])]

testing = combined_ratio_frame_incidents.between_time(start_time, end_time)
testing =  testing[(testing.index.month>9) & (testing.index.month<=12) ]

testing_Clist = testing[list(cross_validated_kappa_SF.keys())]
testing_Clist.columns = list(cross_validated_kappa_SF.keys())

detection_report = []
for column in testing_Clist: #per cluster 
    grouped = testing_Clist[column].groupby([testing_Clist[column].index.hour,
                                             testing_Clist[column].index.minute])
    sm_per_C = safe_margin[column] # safe margin list for each cluster
    kappa = cross_validated_kappa_SF[column]['kappa']
    SF = cross_validated_kappa_SF[column]['SF']
    for key1, group in grouped:
        for index, item in group.iteritems():
            if(pd.isna(item)):continue
            if((item > sm_per_C[kappa]['upper'][key1] ) or (item < sm_per_C[kappa]['lower'][key1] )):
                res_SF = test_residual[column][kappa]
                std_limit = standard_limit_5C_Frame[(standard_limit_5C_Frame['cluster_id']== column) &
                                            (standard_limit_5C_Frame['ka ppa']== kappa) &
                                            (standard_limit_5C_Frame['SF']== SF)]

                index_ar = std_limit.index
                if(res_SF[SF][index] >0):
                    if(res_SF[SF][index]>std_limit.at[index_ar[0],'tau_max']):
                        temp = {'cluster_id':column,'kappa':kappa,'SF':SF,
                                'time':index,'RUC':res_SF[SF][index],'tau_max':std_limit.at[index_ar[0],'tau_max']}
                        detection_report.append(temp)
                else:
                    if(res_SF[SF][index]<std_limit.at[index_ar[0],'tau_min']):
                        temp = {'cluster_id':column,'kappa':kappa,'SF':SF,
                                'time':index,'RUC':res_SF[SF][index],'tau_min':std_limit.at[index_ar[0],'tau_min']}
                        detection_report.append(temp)
detection_report_Frame = pd.DataFrame(detection_report)
detection_report_Frame.set_index('time',inplace = True)

group_detection_report_by_cluster_id = detection_report_Frame.groupby('cluster_id')
actual_detection = []
detection_GT = []
for key,group in group_detection_report_by_cluster_id:
    foucsed_cluster = testing_incident_GT_Clist[testing_incident_GT_Clist['cluster_head']==key]
    for index,row in group.iterrows():
        detection_type = 0
        for index1,row1 in foucsed_cluster.iterrows():
            #iterate only incidents happend for the cluster 
            if((index.month == index1.month) and (index.day == index1.day)):
                #This means incident and detection are on the same day
                if((index.hour >= (index1.hour-2)) & (index.hour <= (index1.hour+2))):
                    #this means successful detection of the incident
                    detection_type = 1
                    temp1 = {'cluster_id':key,'time':index1}
                    detection_GT.append(temp1)
                elif((index.hour >= 6) & (index.hour <= 10) or
                    (index.hour >= 16) & (index.hour <= 18)):
                    #this means detected an incident
                    detection_type = 2
                else:
                    detection_type =3
                break
        temp = {'cluster_id':key,'time':index,'detection_type':detection_type}
        actual_detection.append(temp)

actual_detection_Frame = pd.DataFrame(actual_detection)
actual_detection_Frame.set_index('time',inplace = True)

actual_detection_Frame['detection_number'] = 0
group_actual_detection_Frame = actual_detection_Frame.groupby(['cluster_id'])
for key1, group in group_actual_detection_Frame:
    prev = None
    detection = 0
    group.sort_index(inplace=True)
    for index,item in group.iterrows():
        if((prev == None)):
            prev = index
        else:
            if((prev.month == index.month) & (prev.day == index.day)):
                if(prev.hour == index.hour):
                    diff = index.minute - prev.minute
                    if(diff == 5):
                        group.at[index,'detection_number'] = detection
                        prev = index
                        continue
                    else:
                        detection = detection + 1
                else:
                    H_diff = index.hour - prev.hour
                    if(H_diff == 1):
                        if((index.minute  == 0) & (prev.minute == 55)):
                            group.at[index,'detection_number'] = detection
                            prev = index
                            continue
                        else:
                            detection = detection + 1
                    else:
                        detection = detection + 1
            else: 
                detection = detection + 1
            group.at[index,'detection_number'] = detection
            prev = index
    for index1,item in group.iterrows():
        if(actual_detection_Frame[actual_detection_Frame['cluster_id'] == key1].at[index1,'detection_number'] == 0):
            actual_detection_Frame.at[index1,'detection_number'] = item.detection_number

report = {}
group_by_cluster  = actual_detection_Frame.groupby('cluster_id')
for key, group in group_by_cluster:
    report[key] = {}
    report[key]['cluster_id'] = key
    print('Cluster Id: ',key)
    total_actual_incident = len(testing_incident_GT_Clist[testing_incident_GT_Clist['cluster_head']==key])
    print('Total Actual Incident: ',total_actual_incident)

    report[key]['total_actual_incident'] = total_actual_incident

    group = group[~group.index.duplicated(keep='first')]
    total = len(list(group['detection_number'].unique()))
    incident_frame = testing_incident_GT_Clist[testing_incident_GT_Clist['cluster_head']==key]
    count = 0
    print("incident length: ",len(incident_frame))
    report[key]['incident_frame'] = len(incident_frame)

    temp = group[group['detection_type'] == 1]
    for index,row in incident_frame.iterrows():
        focused_window = temp[(temp.index.month == index.month)&
                                    (temp.index.day == index.day)&
                                    (temp.index.hour >= (index.hour - 2))&
                                    (temp.index.hour <= (index.hour + 2))]
        if(len(focused_window)>0):
            count = count + 1
    detection = len(list(group[group['detection_type'] == 1]['detection_number'].unique()))
    c_detection = len(list(group[group['detection_type'] == 2]['detection_number'].unique()))
    fa_alarm  = total - detection - c_detection
    print('total: ',total,' detection: ',detection,' c_detection: ',c_detection,' fa_alarm: ',fa_alarm,' count: ',count)
    report[key]['results'] = {'total': total, 'detection': detection, 'c_detection': c_detection, 'fa_alarm': fa_alarm, 'count': count}

# pprint(report)
   
# Saving and backing up
fp = os.path.join(results, f'optimized_results_{new_filename}.pkl')
with open(fp, 'wb') as handle:
    pickle.dump(report, handle)
    
fp = os.path.join(results, f'optimized_actual_detection_frame_{new_filename}.pkl')
actual_detection_Frame.to_pickle(fp)

fp = os.path.join(results, f'optimized_detection_report_Frame_{new_filename}.pkl')
detection_report_Frame.to_pickle(fp)

elapsed_time = time.time() - time_start
print(f"Done in {elapsed_time} s")

Cluster Id:  160726844
Total Actual Incident:  9
incident length:  9
total:  666  detection:  26  c_detection:  18  fa_alarm:  622  count:  9
Cluster Id:  429334708
Total Actual Incident:  9
incident length:  9
total:  519  detection:  18  c_detection:  32  fa_alarm:  469  count:  9
Cluster Id:  449617565
Total Actual Incident:  7
incident length:  7
total:  693  detection:  24  c_detection:  30  fa_alarm:  639  count:  6
Cluster Id:  449617633
Total Actual Incident:  10
incident length:  10
total:  576  detection:  38  c_detection:  27  fa_alarm:  511  count:  10
Cluster Id:  1524308595
Total Actual Incident:  8
incident length:  8
total:  567  detection:  30  c_detection:  28  fa_alarm:  509  count:  7
Cluster Id:  1524453968
Total Actual Incident:  6
incident length:  6
total:  605  detection:  23  c_detection:  20  fa_alarm:  562  count:  6
Cluster Id:  1524544056
Total Actual Incident:  6
incident length:  6
total:  563  detection:  19  c_detection:  18  fa_alarm:  526  count:  6
